In [3]:
import pandas as pd

In [16]:
df = pd.read_html('./data/2001Y_1M.xls',encoding='utf-8')
df[0]

,0,1,2,3,4,5,6,7
0,類型代號,基金統編,基金名稱,基金規模,單位淨值,本月定時定額扣款筆數,本月定時定額扣款人數,本月定時定額扣款金額
1,2,00965469,國際第一基金,130247198,8.3200,0,0,0
2,5,00971976,國際萬全基金,727637767,11.8400,3581,3466,16390000
3,4,00992769,國際全球基金,2275475019,31.1700,13420,12604,73741000
4,6,01013101,國際萬寶基金,10753451692,13.2824,0,0,0
5,2,01050148,國際國民基金,605789700,20.2900,25,22,103000
6,2,17606299,國際金龍基金,1150252565,5.2000,423,402,1982000
7,7,20095341,國際萬通基金,3429704001,11.4681,0,0,0
8,3,21100568,國際台灣基金,7451328425,141.2500,0,0,0
9,2,73991751,國際電子基金,1342273871,11.6500,3613,3419,16703000


In [25]:
# bb = pd.ExcelWriter('out.xlsx')
# df[0].to_excel(bb)
# bb.close()